# Работа с реляционными базыми данных

### План
1. Вспоминаем курс БД: Реляционная базы данных и Язык SQL
1. SQLite3
1. PostgreSQL + psycopg2
1. ORM. [SQLAlchemy](http://www.sqlalchemy.org)

### Реляционная базы данных

Реляционные базы данных:
  * Связанная информация, хранящаяся в двумерных таблицах
  * Каждая таблица представляет собой набор полей (колонки) и набор объектов (записей)
  * Порядок записей в таблице произволен, столбцов - фиксирован
  * Идентификация записей осуществляется с помощью первичных ключей

Если коротко, то реляционные базы данных - это таблицы с индексами для быстрого поиска, связанные между собой

### Язык SQL

* [Язык структурированных запросов](https://ru.wikipedia.org/wiki/SQL)
* Стандартизирован, но БД пытаются соответствовать стандарту ANSI не позволяя ему ограничивать их слишком сильно
* Типы: Числа (INTEGER и DECIMAL), строки (CHAR, VARCHAR) и т.д.
* Операторы: SELECT FROM, WHERE, GROUP BY, COUNT, HAVING и т.д.
* Джойны: Inner join, outer join и т.д.
* Подзапросы
* Команды модификации: INSERT, DELETE, UPDATE
* Создание таблиц


### SQLite3

<img src='pictures/sqlite.png' />

* Простейший способ использовать БД
* Вся БД хранится в одном файле
* Отличный способ иметь БД в мобильных приложениях
* Доступ из коммандной строки: sqlite3 filename
* API для Python, с++ и т.д.

#### Попробуем выполнить простой запрос

In [1]:
import sqlite3

# Соединение с базой
conn = sqlite3.connect('example.db')

# Объект для исполнения SQL запросов над базой
cur = conn.cursor()

# Выполним какой-нибудь запрос
cur.execute('select 1')

# Читаем результат. fetchone ожидает единственную строку
print(cur.fetchone())

# Закрываем за собой соединение с базой.
conn.close()

(1,)


Не забываем закрывать соединение! <br>
**Connection - это ресурс, который может и закончиться.** <br>
В этом нам помогут контекстные менеджеры.

In [4]:
# Соединение с базой теперь закроется само.
with sqlite3.connect('example.db') as conn:
    
    # Объект для исполнения SQL запросов над базой
    cur = conn.cursor()

    # Выполним какой-нибудь запрос
    cur.execute("select 'Hello, world!', 6 * 7")

    # Читаем результат. fetchone ожидает единственную строку
    print(cur.fetchone())

('Hello, world!', 42)


Установка соединения может быть дорогой операцией.<br>
Не стоит создавать соединение для каждого запроса. По возможности переиспользуйте старое.

#### Теперь посмотрим как работать с sqlite3 на примере простой базы с пользователями.<br>
Для начала создадим таблицы:

In [4]:
def create_tables(cur, conn):
    # Удалим старые таблицы, если есть.
    cur.execute('DROP TABLE IF EXISTS users')
    cur.execute('DROP TABLE IF EXISTS user_types')
    
    # Пишем SQL запрос для создания таблиц
    cur.execute('''
        CREATE TABLE user_types (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name VARCHAR(255)
        )''')
    cur.execute('''
        CREATE TABLE users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            first_name VARCHAR(255),
            last_name VARCHAR(255),
            active BOOLEAN,
            profile INTEGER,
            FOREIGN KEY(profile) REFERENCES user_types(id)
        )''')
    
    # Отправить текущую транзакцию.
    # Если не сделать, то изменения не будут видны в других connect'ax
    # Закрытие соединения не вызывает commit автоматически!
    conn.commit()

Работать с пустыми таблицами - не очень интересно, давайте заполним их

In [20]:
def add_user_types(cur):
    cur.execute('''INSERT INTO user_types (name) VALUES ('Student')''')
    cur.execute('''INSERT INTO user_types (name) VALUES ('Teacher')''')

def add_teachers(cur):
    cur.execute('''
        INSERT INTO users (first_name, last_name, active, profile) VALUES 
            ('Maxim', 'Popov', '1', 
                (SELECT id FROM user_types WHERE name = 'Teacher'))''')
    cur.execute('''
        INSERT INTO users (first_name, last_name, active, profile) VALUES 
            ('Igor', 'Orlov', '1', 
                (SELECT id FROM user_types WHERE name = 'Teacher'))''')

def add_students(cur):
    cur.execute('''
        INSERT INTO users (first_name, last_name, active, profile) VALUES 
            ('Ivan', 'Ivanov', '0', 
                (SELECT id FROM user_types WHERE name = 'Student'))''')
    cur.execute('''
        INSERT INTO users (first_name, last_name, active, profile) VALUES 
            ('Petr', 'Petrov', '0', 
                (SELECT id FROM user_types WHERE name = 'Student'))''')
    cur.execute('''
        INSERT INTO users (first_name, last_name, active, profile) VALUES 
            ('Petr', 'Sidorov', '0', 
                (SELECT id FROM user_types WHERE name = 'Student'))''')

Выполним запросы к базе

In [21]:
with sqlite3.connect('example.db') as conn:

    cur = conn.cursor()
    
    # Функция создания таблиц коммитит транзакцию сразу.
    create_tables(cur, conn)
    
    # Функции добавления польователей не вызывают commit.
    add_user_types(cur)
    add_teachers(cur)
    add_students(cur)
    
    # Делаем commit для всех запросов добавления.
    # Если забудем - то таблицы останутся пустыми.
    conn.commit()

Попробуем теперь зачитать всех студентов

In [22]:
def select_all_students(conn):
    query = '''SELECT users.first_name, users.last_name 
               FROM users 
               WHERE profile = (SELECT id FROM user_types WHERE name = 'Student')'''
    cur = conn.cursor()
    cur.execute(query)

    # Получить все строчки результата выполнения query
    # Это не всегда быстро, особенно если много результатов
    # Кроме того большое потребление памяти, так как возвращается list
    rows = cur.fetchall() 

    # Как fetchall только задаем сколько хотим получить
    # rows = cur.fetchmany(10) 

    for row in rows:
        first_name, last_name = row
        print(first_name, last_name)
        
with sqlite3.connect('example.db') as conn:
    select_all_students(conn)

Ivan Ivanov
Petr Petrov
Petr Sidorov


In [23]:
def select_all_students(conn):
    query = '''SELECT users.first_name, users.last_name 
               FROM users 
               WHERE profile = (SELECT id FROM user_types WHERE name = 'Student')'''
    cur = conn.cursor()
    cur.execute(query)

    # А лучше просто проитерироваться
    for row in cur:
        first_name, last_name = row
        print(first_name, last_name)
        
with sqlite3.connect('example.db') as conn:
    select_all_students(conn)

Ivan Ivanov
Petr Petrov
Petr Sidorov


Попробуем сделать учителя Maxim Popov студентом

In [24]:
def alter_teacher(conn):
    cur = conn.cursor()
    cur.execute('''UPDATE users SET profile = (SELECT id FROM user_types WHERE name = "Student")
                   WHERE first_name = "Maxim" AND last_name = "Popov"
    ''')

with sqlite3.connect('example.db') as conn:
    alter_teacher(conn)

Посмотрим на результат

In [25]:
with sqlite3.connect('example.db') as conn:
    select_all_students(conn)

Maxim Popov
Ivan Ivanov
Petr Petrov
Petr Sidorov


Создадим функцию, чтобы добавить всех студентов сразу!

In [26]:
def add_students(conn, students):
    queries = []
    
    for student in students:
        query = '''
            INSERT INTO users (first_name, last_name, active, profile) VALUES 
                ("{}", "{}", 0, 
                    (SELECT id FROM user_types WHERE name = "Student"));'''
        queries.append(query.format(*student))
    print(queries)
    cur = conn.cursor()
    
    # Исполняет сразу несколько запросов, разделенных ';'
    cur.executescript(''.join(queries))

In [27]:
with sqlite3.connect('example.db') as conn:
    students = [('Vladimir', 'Vladimirov'), ('Dmitry', 'Dmitriev')]
    add_strudents(conn, students)
    
    # Проверим, что все вставилось.
    select_all_students(conn)

Maxim Popov
Ivan Ivanov
Petr Petrov
Petr Sidorov
Vladimir Vladimirov
Dmitry Dmitriev


Все отлично работает!

In [28]:
with sqlite3.connect('example.db') as conn:
    students = [('cool', 'hacker", 0, 0); drop table users; --')]
    add_students(conn, students)

['\n            INSERT INTO users (first_name, last_name, active, profile) VALUES \n                ("cool", "hacker", 0, 0); drop table users; --", 0, \n                    (SELECT id FROM user_types WHERE name = "Student"));']


OperationalError: near "(": syntax error

In [ ]:
INSERT INTO users (first_name, last_name, active, profile) VALUES
("cool", "hacker", 0, 0); drop table users; --", 0, \n         
(SELECT id FROM user_types WHERE name = "Student"));

In [18]:
with sqlite3.connect('example.db') as conn:
    select_all_students(conn)

OperationalError: no such table: users

Where is my table, Lebowski???

<img src="https://imgs.xkcd.com/comics/exploits_of_a_mom.png">

# Работа с <имя вашей любиной базы>

Почти ничем не отличается от SQLite.*

\* В случае, если поддержан [Python DB API 2.0](https://www.python.org/dev/peps/pep-0249/)

PostgreSQL: [psycopg2](http://initd.org/psycopg/)
<img src="pictures/postgresql.jpg">

In [150]:
try:
    import psycopg2
except:
    !pip install psycopg2-binary

In [151]:
# Поменялся способ подключения. 
# Теперь указываем не файл, а хост, базу, пользователя и пароль.
# Чтобы это работало, должен быть запущен psqld.
# Также, нужно создать пользователя с паролем и базу.
with psycopg2.connect(dbname='testdb', user='testuser', 
                      host='localhost', password='testuser') as conn:
    
    # А остальной интерфейс сохранился.
    cur = conn.cursor()
    cur.execute("select 'Hello, world!', 6 * 7")
    print(cur.fetchone())

('Hello, world!', 42)


[Как установить PostgreSQL в Ubuntu и добавить пользователя](https://www.digitalocean.com/community/tutorials/postgresql-ubuntu-16-04-ru#создание-новой-роли)

Создадим таблички

In [153]:
with psycopg2.connect(dbname='testdb', user='testuser', host='localhost', password='testuser') as conn:
    
    cur = conn.cursor()
    
    # Удалим старые таблицы, если есть.
    cur.execute('DROP TABLE IF EXISTS users')
    cur.execute('DROP TABLE IF EXISTS user_types')
    
    # SQL диалект может немного меняться
    # Используем тип SERIAL вместо свойстав AUTOINCREMENT
    cur.execute('''
        CREATE TABLE user_types (
            id SERIAL PRIMARY KEY,
            name VARCHAR(255)
        )
    ''')
    cur.execute('''
        CREATE TABLE users (
            id SERIAL PRIMARY KEY,
            first_name VARCHAR(255),
            last_name VARCHAR(255),
            active BOOLEAN,
            profile INTEGER,
            FOREIGN KEY(profile) REFERENCES user_types(id)
        )
    ''')

Многие запросы работают без изменений.

In [154]:
with psycopg2.connect(dbname='testdb', user='testuser', host='localhost', password='testuser') as conn:
    cur = conn.cursor()
    
    add_user_types(cur)
    add_teachers(cur)
    add_strudents(cur)
    conn.commit()
    
    select_all_students(conn)

Ivan Ivanov
Petr Petrov
Petr Sidorov


# SQLAlchemy

<img src='pictures/sql_alchemy.png' />

In [29]:
import sqlalchemy
from sqlalchemy import create_engine

In [36]:
engine = create_engine('sqlite:///example_alchemy.db', echo=True)
# engine = create_engine('sqlite:///:memory:', echo=True)

# echo=True - для дополнительного логгирования

#### Опишем данные, которые хотим хранить

In [37]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Boolean, ForeignKey
from sqlalchemy.orm import relationship

Base = declarative_base()

class UserType(Base):
    __tablename__ = 'user_types'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    users = relationship("User")
    
    def __repr__(self):
        return "<UserType(name={})>".format(self.name)  

class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    first_name = Column(String)
    last_name = Column(String)
    active = Column(Boolean)
    profile_id = Column(Integer, ForeignKey('user_types.id'))
    profile = relationship("UserType", back_populates="users")
    
    def __repr__(self):
        return "<User(first_name={}, last_name={}, active={})>"\
            .format(self.first_name, self.last_name, self.active)
    

In [38]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('first_name', String(), table=<users>), Column('last_name', String(), table=<users>), Column('active', Boolean(), table=<users>), Column('profile_id', Integer(), ForeignKey('user_types.id'), table=<users>), schema=None)

#### Создадим таблицы в базе

In [43]:
Base.metadata.create_all(engine)

2019-04-08 19:16:17,818 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("user_types")
2019-04-08 19:16:17,822 INFO sqlalchemy.engine.base.Engine ()
2019-04-08 19:16:17,825 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-04-08 19:16:17,827 INFO sqlalchemy.engine.base.Engine ()
2019-04-08 19:16:17,829 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE user_types (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	PRIMARY KEY (id)
)


2019-04-08 19:16:17,831 INFO sqlalchemy.engine.base.Engine ()
2019-04-08 19:16:17,839 INFO sqlalchemy.engine.base.Engine COMMIT
2019-04-08 19:16:17,841 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	first_name VARCHAR, 
	last_name VARCHAR, 
	active BOOLEAN, 
	profile_id INTEGER, 
	PRIMARY KEY (id), 
	CHECK (active IN (0, 1)), 
	FOREIGN KEY(profile_id) REFERENCES user_types (id)
)


2019-04-08 19:16:17,842 INFO sqlalchemy.engine.base.Engine ()
2019-04-08 19:16:17,851 INFO sqlalchemy.engine.base.Engine COMMIT


Вставим данные

In [44]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

teacher = UserType(name='teacher')
session.add(teacher)

student = UserType(name='student')
session.add(student)

session.commit()

2019-04-08 19:18:11,134 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-08 19:18:11,138 INFO sqlalchemy.engine.base.Engine INSERT INTO user_types (name) VALUES (?)
2019-04-08 19:18:11,141 INFO sqlalchemy.engine.base.Engine ('teacher',)
2019-04-08 19:18:11,143 INFO sqlalchemy.engine.base.Engine INSERT INTO user_types (name) VALUES (?)
2019-04-08 19:18:11,145 INFO sqlalchemy.engine.base.Engine ('student',)
2019-04-08 19:18:11,149 INFO sqlalchemy.engine.base.Engine COMMIT


Найдем тип учителя в базе

In [46]:
#     SELECT * FROM USERTYPE WHERE NAME == "TEACHER" TOP 1  
res = session.query(UserType).filter_by(name='teacher').first() 
res

2019-04-08 19:20:53,744 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.name = ?
 LIMIT ? OFFSET ?
2019-04-08 19:20:53,746 INFO sqlalchemy.engine.base.Engine ('teacher', 1, 0)


<UserType(name=teacher)>

In [47]:
teacher is res

True

one vs first

In [10]:
session.query(UserType).filter_by(name='teacher12').one() 

2019-04-07 10:46:00,036 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.name = ?
2019-04-07 10:46:00,038 INFO sqlalchemy.engine.base.Engine ('teacher12',)


NoResultFound: No row was found for one()

In [11]:
session.query(UserType).one() 

2019-04-07 10:46:00,365 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types
2019-04-07 10:46:00,372 INFO sqlalchemy.engine.base.Engine ()


MultipleResultsFound: Multiple rows were found for one()

Разные виды "select"

In [48]:
results = session.query(UserType).filter_by(name="student").order_by(UserType.id).all()
print(type(results))
print(results)

2019-04-08 19:23:09,372 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.name = ? ORDER BY user_types.id
2019-04-08 19:23:09,374 INFO sqlalchemy.engine.base.Engine ('student',)
<class 'list'>
[<UserType(name=student)>]


In [13]:
for name, user_id in session.query(UserType.name, UserType.id).order_by(UserType.name):
    print(name, user_id)

2019-04-07 10:46:01,275 INFO sqlalchemy.engine.base.Engine SELECT user_types.name AS user_types_name, user_types.id AS user_types_id 
FROM user_types ORDER BY user_types.name
2019-04-07 10:46:01,277 INFO sqlalchemy.engine.base.Engine ()
student 2
teacher 1


#### Добавим нового учителя

In [14]:
user = User(first_name="Maxim", last_name="Popov", active=True)
teacher.users.append(user)
session.add(user)
session.commit()

2019-04-07 10:46:02,287 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.first_name AS users_first_name, users.last_name AS users_last_name, users.active AS users_active, users.profile_id AS users_profile_id 
FROM users 
WHERE ? = users.profile_id
2019-04-07 10:46:02,290 INFO sqlalchemy.engine.base.Engine (1,)
2019-04-07 10:46:02,295 INFO sqlalchemy.engine.base.Engine INSERT INTO users (first_name, last_name, active, profile_id) VALUES (?, ?, ?, ?)
2019-04-07 10:46:02,296 INFO sqlalchemy.engine.base.Engine ('Maxim', 'Popov', 1, 1)
2019-04-07 10:46:02,298 INFO sqlalchemy.engine.base.Engine COMMIT


In [15]:
res = session.query(User).first() 
res

2019-04-07 10:46:03,457 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-07 10:46:03,460 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.first_name AS users_first_name, users.last_name AS users_last_name, users.active AS users_active, users.profile_id AS users_profile_id 
FROM users
 LIMIT ? OFFSET ?
2019-04-07 10:46:03,461 INFO sqlalchemy.engine.base.Engine (1, 0)


<User(first_name=Maxim, last_name=Popov, active=True)>

In [16]:
res.profile

2019-04-07 10:46:06,300 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.id = ?
2019-04-07 10:46:06,306 INFO sqlalchemy.engine.base.Engine (1,)


<UserType(name=teacher)>

Добавим студента

In [17]:
user = User(first_name="Petr", last_name="Sidorov", active=False, profile=student)
session.add(user)
session.commit()

2019-04-07 10:46:06,613 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.id = ?
2019-04-07 10:46:06,619 INFO sqlalchemy.engine.base.Engine (2,)
2019-04-07 10:46:06,624 INFO sqlalchemy.engine.base.Engine INSERT INTO users (first_name, last_name, active, profile_id) VALUES (?, ?, ?, ?)
2019-04-07 10:46:06,629 INFO sqlalchemy.engine.base.Engine ('Petr', 'Sidorov', 0, 2)
2019-04-07 10:46:06,633 INFO sqlalchemy.engine.base.Engine COMMIT


In [18]:
student.users

2019-04-07 10:46:06,774 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-07 10:46:06,777 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.id = ?
2019-04-07 10:46:06,783 INFO sqlalchemy.engine.base.Engine (2,)
2019-04-07 10:46:06,788 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.first_name AS users_first_name, users.last_name AS users_last_name, users.active AS users_active, users.profile_id AS users_profile_id 
FROM users 
WHERE ? = users.profile_id
2019-04-07 10:46:06,789 INFO sqlalchemy.engine.base.Engine (2,)


[<User(first_name=Petr, last_name=Sidorov, active=False)>]

Сделаем его учителем

In [19]:
user.profile = teacher
session.commit()

2019-04-07 10:46:08,056 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.id = ?
2019-04-07 10:46:08,061 INFO sqlalchemy.engine.base.Engine (1,)
2019-04-07 10:46:08,067 INFO sqlalchemy.engine.base.Engine UPDATE users SET profile_id=? WHERE users.id = ?
2019-04-07 10:46:08,072 INFO sqlalchemy.engine.base.Engine (1, 2)
2019-04-07 10:46:08,081 INFO sqlalchemy.engine.base.Engine COMMIT


In [20]:
student.users

2019-04-07 10:46:08,226 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-07 10:46:08,228 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.id = ?
2019-04-07 10:46:08,229 INFO sqlalchemy.engine.base.Engine (2,)
2019-04-07 10:46:08,233 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.first_name AS users_first_name, users.last_name AS users_last_name, users.active AS users_active, users.profile_id AS users_profile_id 
FROM users 
WHERE ? = users.profile_id
2019-04-07 10:46:08,234 INFO sqlalchemy.engine.base.Engine (2,)


[]

In [21]:
teacher.users

2019-04-07 10:46:08,511 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.id = ?
2019-04-07 10:46:08,515 INFO sqlalchemy.engine.base.Engine (1,)
2019-04-07 10:46:08,521 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.first_name AS users_first_name, users.last_name AS users_last_name, users.active AS users_active, users.profile_id AS users_profile_id 
FROM users 
WHERE ? = users.profile_id
2019-04-07 10:46:08,523 INFO sqlalchemy.engine.base.Engine (1,)


[<User(first_name=Maxim, last_name=Popov, active=True)>,
 <User(first_name=Petr, last_name=Sidorov, active=False)>]

Какая еще бывает фильтрация?

In [22]:
# equals
session.query(UserType).filter(UserType.name == 'teacher')
# not equals
session.query(UserType).filter(UserType.name != 'teacher')
# LIKE
session.query(UserType).filter(UserType.name.like('%tea%'))
# ILIKE (case-insensitive LIKE):
session.query(UserType).filter(UserType.name.ilike('%TeA%'))
# IS NULL
session.query(UserType).filter(UserType.name.is_(None))
# IS NOT NULL
session.query(UserType).filter(UserType.name.isnot(None))
              
from sqlalchemy import and_
#AND
session.query(UserType).filter(and_(UserType.name.isnot(None), UserType.name != 'abc'))
session.query(UserType).filter(UserType.name.isnot(None), UserType.name != 'abc')
session.query(UserType).filter(UserType.name.isnot(None)).filter(UserType.name != 'abc')

#OR
from sqlalchemy import or_
session.query(UserType).filter(or_(UserType.name.isnot(None), UserType.name != 'abc'))

In [23]:
#IN
session.query(User).filter(User.profile_id.in_(
    session.query(UserType.id)
)).all()

2019-04-07 10:46:12,259 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.first_name AS users_first_name, users.last_name AS users_last_name, users.active AS users_active, users.profile_id AS users_profile_id 
FROM users 
WHERE users.profile_id IN (SELECT user_types.id AS user_types_id 
FROM user_types)
2019-04-07 10:46:12,262 INFO sqlalchemy.engine.base.Engine ()


[<User(first_name=Maxim, last_name=Popov, active=True)>,
 <User(first_name=Petr, last_name=Sidorov, active=False)>]